<a href="https://colab.research.google.com/github/jvataidee/PesquisaOperacional/blob/master/maxloglihood_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.8 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm

In [38]:
# Definindo as ações de três empresas como alternativas de projetos
tickers = ['AAPL', 'GOOGL', 'AMZN', "ITUB", "TSLA"]

# Baixando os dados históricos das ações do Yahoo Finance
data = yf.download(tickers, start='2010-01-01', end='2023-03-16')['Adj Close']
data.head()

[*********************100%***********************]  5 of 5 completed


,AAPL,AMZN,GOOGL,ITUB,TSLA
Date,,,,,
2010-01-04,6.505280,6.6950,15.684434,5.503197,NaN
2010-01-05,6.516528,6.7345,15.615365,5.521842,NaN
2010-01-06,6.412874,6.6125,15.221722,5.454249,NaN
2010-01-07,6.401017,6.5000,14.867367,5.333046,NaN
2010-01-08,6.443573,6.6760,15.065566,5.307406,NaN


In [39]:
# Calculando os retornos diários das ações
returns = data.pct_change().dropna()

# Calculando as médias e desvios padrão dos retornos diários
means = returns.mean()
std_devs = returns.std()

In [40]:
means

AAPL     0.001093
AMZN     0.001115
GOOGL    0.000814
ITUB     0.000333
TSLA     0.002130
dtype: float64

In [41]:
std_devs

AAPL     0.017984
AMZN     0.020829
GOOGL    0.017166
ITUB     0.025453
TSLA     0.036165
dtype: float64

In [42]:
def max_log_likelihood(mean, std_dev, data):
    log_likelihood = 0
    for value in data:
        log_likelihood += np.log(norm.pdf(value, mean, std_dev))
    return log_likelihood

In [43]:
# Calculando as probabilidades com base na máxima verossimilhança
probabilities = []
for ticker in tickers:
    probabilities.append(max_log_likelihood(means[ticker], std_devs[ticker], returns[ticker]))

In [44]:
# Normalizando as probabilidades para que somem 1
probabilities = np.array(probabilities)
probabilities /= probabilities.sum()

# Imprimindo as probabilidades
for ticker, prob in zip(tickers, probabilities):
    print(f'Probabilidade para {ticker}: {prob:.4f}')

# Selecionando a ação com maior probabilidade
best_project = tickers[np.argmax(probabilities)]
print(f'\nO projeto com maior probabilidade de sucesso é: {best_project}')


Probabilidade para AAPL: 0.2193
Probabilidade para GOOGL: 0.2233
Probabilidade para AMZN: 0.2070
Probabilidade para ITUB: 0.1900
Probabilidade para TSLA: 0.1604

O projeto com maior probabilidade de sucesso é: GOOGL
